ИМПОРТ БИБЛИОТЕК

In [1]:

import scipy
import matplotlib
import sklearn
import pandas as pd 
import numpy as np #
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler, RobustScaler, Normalizer, LabelEncoder, StandardScaler
from sklearn.tree import DecisionTreeClassifier 
from sklearn import tree 
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV 
from sklearn.metrics import accuracy_score 
from sklearn.cluster import KMeans 


from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from ydata_profiling import ProfileReport
%matplotlib inline